In [1]:
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    
    get_folders_by_plate_id,
)
from time import time_ns
from amftrack.util.dbx import upload_folders, load_dbx, download, get_dropbox_folders
from datetime import datetime

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
all_folders_drop = get_dropbox_folders("/DATA/PRINCE", True)


2022-08-08 18:31:49,053-[INFO]- dropbox:388 -> Refreshing access token.
2022-08-08 18:31:49,216-[INFO]- dropbox:477 -> Request to files/list_folder
2022-08-08 18:31:51,901-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-08-08 18:31:53,261-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-08-08 18:31:54,251-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-08-08 18:31:55,196-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-08-08 18:31:56,328-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-08-08 18:31:57,259-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-08-08 18:31:58,446-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-08-08 18:31:59,512-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-08-08 18:32:00,949-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-08-08 18:32:01,997-[INFO]- dropbox:477 -> Request to files/list_folder/continue
20

In [9]:
on_dropbox = all_folders_drop.loc[all_folders_drop['CrossDate'].between("20220520","20220523")]
plates = on_dropbox['unique_id'].unique()


In [22]:
myr_plates = [str(plate) for plate in [511,510,514,508]]

In [25]:
on_dropbox = all_folders_drop.loc[all_folders_drop['Plate'].isin(myr_plates)]
plates = on_dropbox['unique_id'].unique()


In [26]:
plates

array(['508_20220623', '510_20220623', '511_20220623', '514_20220623'],
      dtype=object)

In [4]:
directory_targ = directory_project
directory_targ = os.path.join(directory_scratch,'stitch_temp')+'/'


In [5]:
plates

array(['1030_20220523', '1037_20220523', '1014_20220525', '19_20220525',
       '1042_20220523', '22_20220522', '1061_20220523', '1028_20220523'],
      dtype=object)

In [10]:
on_dropbox

,folder,size,change_date,tot_path_drop,Plate,CrossDate,unique_id
422,20220722_0642_Plate27,NaN,2022-07-31 17:08:30,data/prince/1030_20220523/20220722_0642_plate27,1030,20220523,1030_20220523
469,20220721_2242_Plate27,NaN,2022-07-31 15:23:29,data/prince/1030_20220523/20220721_2242_plate27,1030,20220523,1030_20220523
516,20220721_1442_Plate27,NaN,2022-07-31 13:42:59,data/prince/1030_20220523/20220721_1442_plate27,1030,20220523,1030_20220523
545,20220721_0941_Plate27,NaN,2022-07-31 12:39:42,data/prince/1030_20220523/20220721_0941_plate27,1030,20220523,1030_20220523
562,20220721_0541_Plate27,NaN,2022-07-31 12:00:24,data/prince/1030_20220523/20220721_0541_plate27,1030,20220523,1030_20220523
...,...,...,...,...,...,...,...
18485,20220530_0236_Plate04,NaN,2022-06-15 08:13:48,data/prince/1061_20220523/20220530_0236_plate04,1061,20220523,1061_20220523
18486,20220527_0941_Plate04,NaN,2022-06-15 08:11:17,data/prince/1061_20220523/20220527_0941_plate04,1061,20220523,1061_20220523
18489,20220525_0413_Plate09,NaN,2022-06-15 08:02:35,data/prince/1028_20220523/20220525_0413_plate09,1028,20220523,1028_20220523
18512,20220527_1157_Plate09,NaN,2022-06-15 06:59:17,data/prince/1028_20220523/20220527_1157_plate09,1028,20220523,1028_20220523


In [7]:
from time import sleep
sleep(60)

In [27]:
name_job = f"one_shot_{time_ns()}.sh"
run_launcher('dropbox_downloader.py',[directory_targ,name_job,4],plates,'12:00:00',name_job=name_job)
# run_launcher('dropbox_uploader.py',[directory_targ,name_job],plates,'40:00',dependency=True,name_job = name_job)


Submitted batch job 1317548


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 245760 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


In [4]:
run_launcher('skeletonizer.py',[directory_targ],plates,'20:00',dependency=True)

Submitted batch job 1129540


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 245760 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


In [9]:
name_job = f"one_shot_{time_ns()}.sh"

run_launcher('width_extracter.py',[directory_targ,name_job,10],plates,'12:00:00',dependency=True,name_job=name_job)

Submitted batch job 1314729


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 245760 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


In [ ]:
run_launcher('hypha_identifier.py',[directory_targ,name_job,10],plates,'20:00',dependency=True,name_job=name_job)

In [3]:
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)

analysed:   0%|          | 0/841 [00:00<?, ?it/s]

In [6]:
all_folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp,unique_id,datetime
0,/scratch-shared/amftrack/stitch_temp/20220512_...,1045,2,'Carrot','C2','001P100N200C','Y','-',20220504,25,...,False,False,False,False,False,"12.05.2022, 17:43:",20220512_1743_Plate02,NaN,1045_20220504,2022-05-12 17:43:00
1,/scratch-shared/amftrack/stitch_temp/20220606_...,4,26,'Carrot','Agg','001P100N100C','Y','-',20220419,25,...,False,True,True,True,True,"06.06.2022, 00:15:",20220606_0015_Plate26,NaN,4_20220419,2022-06-06 00:15:00
2,/scratch-shared/amftrack/stitch_temp/20220514_...,1015,1,'Carrot','C2','001P100N200C','Y','-',20220504,25,...,False,False,False,False,False,"14.05.2022, 15:46:",20220514_1546_Plate01,NaN,1015_20220504,2022-05-14 15:46:00
3,/scratch-shared/amftrack/stitch_temp/20220517_...,1023,15,'Carrot','C2','001P100N200C','Y','-',20220502,25,...,False,False,False,False,False,"17.05.2022, 15:29:",20220517_1529_Plate15,NaN,1023_20220502,2022-05-17 15:29:00
4,/scratch-shared/amftrack/stitch_temp/20220518_...,33,7,'Carrot','Agg','001P100N100C','Y','-',20220502,25,...,False,False,False,False,False,"18.05.2022, 13:17:",20220518_1317_Plate07,NaN,33_20220502,2022-05-18 13:17:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025,/scratch-shared/amftrack/stitch_temp/20220521_...,1033,28,'Carrot','C2','001P100N200C','Y','-',20220502,25,...,False,False,False,False,False,"21.05.2022, 00:15:",20220521_0015_Plate28,NaN,1033_20220502,2022-05-21 00:15:00
1026,/scratch-shared/amftrack/stitch_temp/20220511_...,1023,15,'Carrot','C2','001P100N200C','Y','-',20220502,25,...,False,False,False,False,False,"11.05.2022, 02:44:",20220511_0244_Plate15,NaN,1023_20220502,2022-05-11 02:44:00
1027,/scratch-shared/amftrack/stitch_temp/20220510_...,33,7,'Carrot','Agg','001P100N100C','Y','-',20220502,25,...,False,False,False,False,False,"10.05.2022, 09:59:",20220510_0959_Plate07,NaN,33_20220502,2022-05-10 09:59:00
1028,/scratch-shared/amftrack/stitch_temp/20220517_...,1040,22,'Carrot','A5','001P100N100C','Y','-',20220502,25,...,False,False,False,False,False,"17.05.2022, 13:48:",20220517_1348_Plate22,NaN,1040_20220502,2022-05-17 13:48:00


In [4]:
# plates= ['12_20220502']
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
# folders = all_folders.loc[all_folders["/Analysis/nx_graph_pruned_labeled.p"]==True]
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp,unique_id,datetime
0,/scratch-shared/amftrack/stitch_temp/20220509_...,1053,39,'Carrot','C2','001P100N200C','Y','-',20220413,25,...,False,True,True,True,True,"09.05.2022, 05:50:",20220509_0550_Plate39,NaN,1053_20220413,2022-05-09 05:50:00
1,/scratch-shared/amftrack/stitch_temp/20220504_...,1053,39,'Carrot','C2','001P100N200C','Y','-',20220413,25,...,False,True,True,True,True,"04.05.2022, 20:20:",20220504_2020_Plate39,NaN,1053_20220413,2022-05-04 20:20:00
5,/scratch-shared/amftrack/stitch_temp/20220418_...,1053,7,'Carrot','C2','001P100N200C','Y','-',20220413,25,...,False,True,True,True,True,"18.04.2022, 19:45:",20220418_1945_Plate07,NaN,1053_20220413,2022-04-18 19:45:00
9,/scratch-shared/amftrack/stitch_temp/20220426_...,1053,7,'Carrot','C2','001P100N200C','Y','-',20220413,25,...,False,True,True,True,True,"26.04.2022, 20:36:",20220426_2036_Plate07,NaN,1053_20220413,2022-04-26 20:36:00
13,/scratch-shared/amftrack/stitch_temp/20220421_...,1053,7,'Carrot','C2','001P100N200C','Y','-',20220413,25,...,False,True,True,True,True,"21.04.2022, 12:53:",20220421_1253_Plate07,NaN,1053_20220413,2022-04-21 12:53:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803,/scratch-shared/amftrack/stitch_temp/20220512_...,1053,39,'Carrot','C2','001P100N200C','Y','-',20220413,25,...,False,True,True,True,True,"12.05.2022, 04:51:",20220512_0451_Plate39,NaN,1053_20220413,2022-05-12 04:51:00
806,/scratch-shared/amftrack/stitch_temp/20220428_...,1053,7,'Carrot','C2','001P100N200C','Y','-',20220413,25,...,True,True,True,True,True,"28.04.2022, 04:36:",20220428_0436_Plate07,NaN,1053_20220413,2022-04-28 04:36:00
807,/scratch-shared/amftrack/stitch_temp/20220501_...,1053,7,'Carrot','C2','001P100N200C','Y','-',20220413,25,...,False,True,True,True,True,"01.05.2022, 22:58:",20220501_2258_Plate07,NaN,1053_20220413,2022-05-01 22:58:00
808,/scratch-shared/amftrack/stitch_temp/20220420_...,1053,7,'Carrot','C2','001P100N200C','Y','-',20220413,25,...,False,True,True,True,True,"20.04.2022, 22:54:",20220420_2254_Plate07,NaN,1053_20220413,2022-04-20 22:54:00


In [7]:
all_folders['unique_id'].unique()

array(['1053_20220413', '1040_20220502', '4_20220419', '1023_20220502',
       '94_20201123', '938_20220325', '961_20220328'], dtype=object)

In [13]:
 i= 0
unique_ids = list(set(folders["unique_id"].values))
unique_ids.sort()
select = folders.loc[folders["unique_id"] == unique_ids[i]]

In [8]:
all_folders
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime
0,/scratch-shared/amftrack/stitch_temp/20220513_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"13.05.2022, 14:35:",20220513_1435_Plate17,6_20220509,2022-05-13 14:35:00
1,/scratch-shared/amftrack/stitch_temp/20220517_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"17.05.2022, 19:35:",20220517_1935_Plate17,6_20220509,2022-05-17 19:35:00
2,/scratch-shared/amftrack/stitch_temp/20220511_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"11.05.2022, 19:50:",20220511_1950_Plate17,6_20220509,2022-05-11 19:50:00
3,/scratch-shared/amftrack/stitch_temp/20220517_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"17.05.2022, 06:56:",20220517_0656_Plate17,6_20220509,2022-05-17 06:56:00
4,/scratch-shared/amftrack/stitch_temp/20220513_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"13.05.2022, 10:35:",20220513_1035_Plate17,6_20220509,2022-05-13 10:35:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,/scratch-shared/amftrack/stitch_temp/20220513_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"13.05.2022, 04:32:",20220513_0432_Plate17,6_20220509,2022-05-13 04:32:00
67,/scratch-shared/amftrack/stitch_temp/20220517_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"17.05.2022, 17:35:",20220517_1735_Plate17,6_20220509,2022-05-17 17:35:00
68,/scratch-shared/amftrack/stitch_temp/20220520_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"20.05.2022, 15:37:",20220520_1537_Plate17,6_20220509,2022-05-20 15:37:00
69,/scratch-shared/amftrack/stitch_temp/20220520_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"20.05.2022, 13:37:",20220520_1337_Plate17,6_20220509,2022-05-20 13:37:00


In [5]:
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)
# folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
# folders = all_fold.loc[folders["/Analysis/skeleton_pruned_compressed.mat"]==True]
# folders.iloc[0]['total_path']

analysed:   0%|          | 0/1740 [00:00<?, ?it/s]

In [12]:
all_folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime
0,/scratch-shared/amftrack/stitch_temp/20220524_...,1071,16,'Carrot','A5','001P100N200C','Y','-',20220517,25,...,False,False,False,False,False,False,"24.05.2022, 00:07:",20220524_0007_Plate16,1071_20220517,2022-05-24 00:07:00
1,/scratch-shared/amftrack/stitch_temp/20220606_...,1051,13,'Carrot','C2','001P100N200C','Y','-',20220518,25,...,True,False,True,True,True,True,"06.06.2022, 07:37:",20220606_0737_Plate13,1051_20220518,2022-06-06 07:37:00
2,/scratch-shared/amftrack/stitch_temp/20220526_...,1035,15,'Carrot','C2','001P100N200C','Y','-',20220518,25,...,True,False,True,True,True,True,"26.05.2022, 04:14:",20220526_0414_Plate15,1035_20220518,2022-05-26 04:14:00
3,/scratch-shared/amftrack/stitch_temp/20220528_...,1071,16,'Carrot','A5','001P100N200C','Y','-',20220517,25,...,True,False,True,True,True,True,"28.05.2022, 11:12:",20220528_1112_Plate16,1071_20220517,2022-05-28 11:12:00
4,/scratch-shared/amftrack/stitch_temp/20220604_...,1051,13,'Carrot','C2','001P100N200C','Y','-',20220518,25,...,True,False,True,True,True,True,"04.06.2022, 11:37:",20220604_1137_Plate13,1051_20220518,2022-06-04 11:37:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,/scratch-shared/amftrack/stitch_temp/20220528_...,1039,18,'Carrot','C2','001P100N200C','Y','-',20220516,25,...,True,False,True,True,True,True,"28.05.2022, 15:19:",20220528_1519_Plate18,1039_20220516,2022-05-28 15:19:00
1724,/scratch-shared/amftrack/stitch_temp/20220521_...,1051,13,'Carrot','C2','001P100N200C','Y','-',20220518,25,...,True,False,True,True,True,True,"21.05.2022, 07:24:",20220521_0724_Plate13,1051_20220518,2022-05-21 07:24:00
1725,/scratch-shared/amftrack/stitch_temp/20220605_...,1035,15,'Carrot','C2','001P100N200C','Y','-',20220518,25,...,True,False,True,True,True,True,"05.06.2022, 07:43:",20220605_0743_Plate15,1035_20220518,2022-06-05 07:43:00
1726,/scratch-shared/amftrack/stitch_temp/20220602_...,1051,13,'Carrot','C2','001P100N200C','Y','-',20220518,25,...,True,False,True,True,True,True,"02.06.2022, 05:47:",20220602_0547_Plate13,1051_20220518,2022-06-02 05:47:00


In [25]:
folders = folders.iloc[0:2]

In [6]:
plates = all_folders['unique_id'].unique()

# run_launcher('dropbox_uploader.py',[directory_targ,name_job],plates,'40:00',dependency=True,name_job = name_job)


In [13]:
os.listdir('/scratch-shared/amftrack/stitch_temp/20220606_0737_Plate13')

['param.m', 'Analysis', 'Img', 'StitchedImage.tif', 'PrinceMeasurement.csv']

In [7]:
plates

array(['1071_20220517', '1051_20220518', '1035_20220518', '1039_20220516',
       '1025_20220518', '1047_20220516', '1024_20220516'], dtype=object)

In [8]:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders.loc[folders['/Img/TileConfiguration.txt.registered']==True]

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime
3,/scratch-shared/amftrack/stitch_temp/20220407_...,889,33,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"07.04.2022, 21:55:",20220407_2155_Plate33,889_20220325,2022-04-07 21:55:00
18,/scratch-shared/amftrack/stitch_temp/20220327_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"27.03.2022, 15:35:",20220327_1535_Plate14,889_20220325,2022-03-27 15:35:00
30,/scratch-shared/amftrack/stitch_temp/20220330_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"30.03.2022, 21:40:",20220330_2140_Plate14,889_20220325,2022-03-30 21:40:00
39,/scratch-shared/amftrack/stitch_temp/20220419_...,889,33,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"19.04.2022, 01:20:",20220419_0120_Plate33,889_20220325,2022-04-19 01:20:00
62,/scratch-shared/amftrack/stitch_temp/20220328_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"28.03.2022, 05:36:",20220328_0536_Plate14,889_20220325,2022-03-28 05:36:00
64,/scratch-shared/amftrack/stitch_temp/20220415_...,889,33,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"15.04.2022, 05:55:",20220415_0555_Plate33,889_20220325,2022-04-15 05:55:00
86,/scratch-shared/amftrack/stitch_temp/20220401_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"01.04.2022, 18:45:",20220401_1845_Plate14,889_20220325,2022-04-01 18:45:00
98,/scratch-shared/amftrack/stitch_temp/20220330_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"30.03.2022, 23:40:",20220330_2340_Plate14,889_20220325,2022-03-30 23:40:00
101,/scratch-shared/amftrack/stitch_temp/20220329_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"29.03.2022, 21:10:",20220329_2110_Plate14,889_20220325,2022-03-29 21:10:00
112,/scratch-shared/amftrack/stitch_temp/20220418_...,889,33,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"18.04.2022, 01:20:",20220418_0120_Plate33,889_20220325,2022-04-18 01:20:00


In [12]:
from amftrack.pipeline.launching.run_super import run_parallel,run_launcher

for plate in plates:
    select = folders.loc[folders["Plate"] == plate]
    num_parallel = 128
    time = "12:00:00"
    thresh = 10000  # For R. irregularis, thresh 10000 is good. For Aggregatum, higher may be necessary
    args = [thresh, directory_targ]
    run_parallel(
        "final_alignment.py",
        args,
        select,
        num_parallel,
        time,
        "realign",
        cpus=128,
        node="fat",
        name_job = name_job

    )

Submitted batch job 1138954


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [20]:
plates

['6_20220509']

In [19]:
select

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime


In [13]:
i= 0 
run_info = folders
folder_list = list(run_info["folder"])
folder_list.sort()

dates_datetime_chosen = folder_list[i : i + 2]
print("========")
print(f"Matching plate {dates_datetime_chosen[0]} at dates {dates_datetime_chosen}")
print("========")
dates = dates_datetime_chosen

Matching plate 20220510_1851_Plate17 at dates ['20220510_1851_Plate17', '20220511_1750_Plate17']


In [17]:
from pymatreader import read_mat

from amftrack.pipeline.functions.image_processing.extract_graph import (
    sparse_to_doc,
    get_degree3_nodes,
)
import open3d as o3d
from cycpd import rigid_registration
import pandas as pd
import os
dilateds = []
skels = []
skel_docs = []
for directory_name in dates:
    path_snap = os.path.join(directory_targ, directory_name)
    skel_info = read_mat(path_snap + "/Analysis/skeleton_pruned.mat")
    skel = skel_info["skeleton"]
    skels.append(skel)
    skel_doc = sparse_to_doc(skel)
    skel_docs.append(skel_doc)

In [18]:
skel_docs

[{(9706, 41696): 1,
  (9706, 41697): 1,
  (9706, 41698): 1,
  (9707, 41694): 1,
  (9707, 41695): 1,
  (9708, 41692): 1,
  (9708, 41693): 1,
  (9708, 41694): 1,
  (9709, 41690): 1,
  (9709, 41691): 1,
  (9709, 41692): 1,
  (9710, 41690): 1,
  (9711, 41688): 1,
  (9711, 41689): 1,
  (9712, 41685): 1,
  (9712, 41686): 1,
  (9712, 41687): 1,
  (9713, 41682): 1,
  (9713, 41683): 1,
  (9713, 41684): 1,
  (9714, 41679): 1,
  (9714, 41680): 1,
  (9714, 41681): 1,
  (9715, 41677): 1,
  (9715, 41678): 1,
  (9716, 41675): 1,
  (9716, 41676): 1,
  (9716, 41677): 1,
  (9717, 41674): 1,
  (9717, 41675): 1,
  (9718, 41673): 1,
  (9718, 41674): 1,
  (9719, 41672): 1,
  (9720, 41670): 1,
  (9720, 41671): 1,
  (9721, 41666): 1,
  (9721, 41667): 1,
  (9721, 41668): 1,
  (9721, 41669): 1,
  (9722, 41663): 1,
  (9722, 41664): 1,
  (9722, 41665): 1,
  (9723, 41660): 1,
  (9723, 41661): 1,
  (9723, 41662): 1,
  (9724, 41658): 1,
  (9724, 41659): 1,
  (9725, 41655): 1,
  (9725, 41656): 1,
  (9725, 41657): 1,
